In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df= pd.read_csv('/content/diabetes.csv')

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [ ]:
X = df.iloc[:, :-1].values   # All columns except last → features
y = df.iloc[:, -1].values    # Only last column → target

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [ ]:
X = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [ ]:
model = Sequential([
    Dense(16, activation='relu', input_shape=(8,)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')   # Important: 1 neuron
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=100,
    validation_data=(X_test, y_test)
)

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7530 - loss: 0.4974 - val_accuracy: 0.8052 - val_loss: 0.5090
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7946 - loss: 0.4722 - val_accuracy: 0.8117 - val_loss: 0.5058
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7866 - loss: 0.4673 - val_accuracy: 0.8052 - val_loss: 0.5026
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7738 - loss: 0.4622 - val_accuracy: 0.8052 - val_loss: 0.5002
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7775 - loss: 0.4668 - val_accuracy: 0.8117 - val_loss: 0.4987
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7728 - loss: 0.4634 - val_accuracy: 0.8182 - val_loss: 0.4967
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7783 - loss: 0.4565 - val_accuracy: 0.8117 - val_loss: 0.4941
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7878 - loss: 0.4494 - val_accuracy: 0.8117 -

In [ ]:
# 1. How to choose an appropriate optimizer
# 2. How to determine the number of neurons in a layer
# 3. How to determine the number of layers
# 4. Combining everything into a single model

1. How to choose an appropriate optimizer

In [ ]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 1.7 MB/s eta 0:00:00


In [ ]:
import kerastuner as kt

/tmp/ipython-input-1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
def build_model(hp):

  model = Sequential()
  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.7662337422370911
Total elapsed time: 00h 00m 12s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [ ]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train, y_train,batch_size=32, epochs=100,initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7165 - loss: 0.5582 - val_accuracy: 0.7662 - val_loss: 0.5266
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7382 - loss: 0.5133 - val_accuracy: 0.7857 - val_loss: 0.5070
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7529 - loss: 0.5141 - val_accuracy: 0.7857 - val_loss: 0.4934
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7698 - loss: 0.4970 - val_accuracy: 0.7857 - val_loss: 0.4814
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7673 - loss: 0.4857 - val_accuracy: 0.7987 - val_loss: 0.4743
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7857 - loss: 0.4780 - val_accuracy: 0.7987 - val_loss: 0.4677
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7617 - loss: 0.4791 - val_accuracy: 0.7922 - val_loss: 0.4620
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7726 - loss: 0.4696 - val_accuracy

2. How to determine the number of neurons in a layer


In [ ]:
def build_model(hp):

  model = Sequential()

  units = hp.Int('units', min_value=8, max_value=128)

  model.add(Dense(units = units,activation='relu',input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory = 'mydir',
                        project_name='Veer')

In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.6623376607894897

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 15s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'units': 65}

In [ ]:
model = tuner.get_best_models(num_models = 1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(X_train, y_train ,batch_size=32, epochs=100,initial_epoch=6 ,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9968 - loss: 0.0109 - val_accuracy: 0.7727 - val_loss: 1.4991
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9994 - loss: 0.0080 - val_accuracy: 0.7597 - val_loss: 1.5101
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0056 - val_accuracy: 0.7597 - val_loss: 1.5211
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0059 - val_accuracy: 0.7597 - val_loss: 1.5336
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0044 - val_accuracy: 0.7662 - val_loss: 1.5407
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0053 - val_accuracy: 0.7727 - val_loss: 1.5530
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0057 - val_accuracy: 0.7727 - val_loss: 1.5658
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0038 - val_accuracy: 0.77


3. How to determine the number of layers


In [ ]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(65,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1, max_value=10)):

    model.add(Dense(65,activation='relu'))

  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model



In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory = 'mydir',
                        project_name='num_layers')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 25s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(X_train, y_train ,batch_size=32, epochs=100,initial_epoch=6,validation_data=(X_test,y_test) )

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.7333 - loss: 0.5058 - val_accuracy: 0.8247 - val_loss: 0.4842
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7482 - loss: 0.4806 - val_accuracy: 0.8182 - val_loss: 0.4827
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7918 - loss: 0.4273 - val_accuracy: 0.7987 - val_loss: 0.4859
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7864 - loss: 0.4513 - val_accuracy: 0.7987 - val_loss: 0.4758
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7797 - loss: 0.4361 - val_accuracy: 0.7987 - val_loss: 0.4785
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8126 - loss: 0.4329 - val_accuracy: 0.7857 - val_loss: 0.4920
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7618 - loss: 0.4375 - val_accuracy: 0.8052 - val_loss: 0.4749
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8307 - loss: 0.3939 - val_accuracy


4. Combining everything into a single model

In [ ]:
def build_model(hp):
    model = Sequential()

    num_layers = hp.Int('num_layers', min_value=1, max_value=10)

    for i in range(num_layers):
        units = hp.Int(f'units_{i}', min_value=8, max_value=128, step=8)
        activation = hp.Choice(f'activation_{i}', values=['relu', 'tanh', 'sigmoid'])

        if i == 0:
            model.add(Dense(units=units, activation=activation, input_shape=(8,)))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(Dense(units=units, activation=activation))
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))


    model.add(Dense(1, activation='sigmoid'))

    optimizer = hp.Choice(
        'optimizer',
        values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']
    )

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='final1'
                        )

In [ ]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 09s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 20s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9,
 'units_0': 128,
 'activation_0': 'sigmoid',
 'dropout0': 0.3,
 'optimizer': 'rmsprop',
 'units_1': 8,
 'activation_1': 'relu',
 'dropout1': 0.1,
 'units_2': 8,
 'activation_2': 'relu',
 'dropout2': 0.1,
 'units_3': 8,
 'activation_3': 'relu',
 'dropout3': 0.1,
 'units_4': 8,
 'activation_4': 'relu',
 'dropout4': 0.1,
 'units_5': 8,
 'activation_5': 'relu',
 'dropout5': 0.1,
 'units_6': 8,
 'activation_6': 'relu',
 'dropout6': 0.1,
 'units_7': 8,
 'activation_7': 'relu',
 'dropout7': 0.1,
 'units_8': 8,
 'activation_8': 'relu',
 'dropout8': 0.1}

In [ ]:
model = tuner.get_best_models(num_models= 1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit(X_train,y_train,epochs=200,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.6482 - loss: 0.6731 - val_accuracy: 0.6429 - val_loss: 0.6649
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6305 - loss: 0.6635 - val_accuracy: 0.6429 - val_loss: 0.6511
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6256 - loss: 0.6629 - val_accuracy: 0.6429 - val_loss: 0.6486
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6310 - loss: 0.6526 - val_accuracy: 0.6429 - val_loss: 0.6356
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6519 - loss: 0.6340 - val_accuracy: 0.6429 - val_loss: 0.6252
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6531 - loss: 0.6245 - val_accuracy: 0.6429 - val_loss: 0.6124
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6463 - loss: 0.6309 - val_accuracy: 0.6429 - val_loss: 0.6102
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6649 - loss: 0.5963 - val_accuracy: 0.64

In [ ]:
#val_accuracy=79,max=81